<a href="https://colab.research.google.com/github/Hafsa620/PAI-LAB/blob/main/BERT_Feature_Extractor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Sentiment Analysis (Text Classification)

1. Downloading Datset from Kaggle to Google Colab
2. Text Cleaning
3. BERT Model (Feature Engineering)
4. ML Model

In [1]:
!pip install kaggle
import os
import json

with open('/content/kaggle.json') as f:
  kaggle_json=json.load(f)

os.environ['KAGGLE_USERNAME']=kaggle_json['username']
os.environ['KAGGLE_KEY']=kaggle_json['key']

!kaggle datasets download lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
!unzip imdb-dataset-of-50k-movie-reviews.zip

Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
License(s): other
 82% 21.0M/25.7M [00:00<00:00, 35.8MB/s]
100% 25.7M/25.7M [00:01<00:00, 26.2MB/s]
Archive:  imdb-dataset-of-50k-movie-reviews.zip
  inflating: IMDB Dataset.csv        


In [2]:
!pip install -U "tensorflow-text==2.17.*"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 28.4 MB/s eta 0:00:00


Importing preprocessing libraries

In [3]:
import pandas as pd
import re
import nltk
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

nltk.download('stopwords')
nltk.download('punkt_tab')
nltk.download('wordnet')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

Loading Dataset

In [4]:
data=pd.read_csv('IMDB Dataset.csv')
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [5]:
df=data.iloc[:1500]

Text cleaning

In [6]:
#removing urls
def urls(text):
  url=re.compile(r'https?://\S+|www\.\S+')
  return url.sub(r'',text)

#removing html tags
def html(text):
  html=re.compile(r'<.*?>')
  return html.sub(r'',text)

#removing special characters
def remove_special_char(text):
  text=re.sub('[^a-zA-Z0-9\s]','',text)
  return text
#extra spaces
def remove_extra_space(text):
  text=re.sub('\s+',' ',text)
  return text

In [7]:
df['review']=df['review'].apply(urls)
df['review']=df['review'].apply(html)
df['review']=df['review'].apply(remove_special_char)
df['review']=df['review'].apply(remove_extra_space)


<ipython-input-7-fdddf0c627f7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review']=df['review'].apply(urls)
<ipython-input-7-fdddf0c627f7>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review']=df['review'].apply(html)
<ipython-input-7-fdddf0c627f7>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide

In [8]:
df['review'][1]

'A wonderful little production The filming technique is very unassuming very oldtimeBBC fashion and gives a comforting and sometimes discomforting sense of realism to the entire piece The actors are extremely well chosen Michael Sheen not only has got all the polari but he has all the voices down pat too You can truly see the seamless editing guided by the references to Williams diary entries not only is it well worth the watching but it is a terrificly written and performed piece A masterful production about one of the great masters of comedy and his life The realism really comes home with the little things the fantasy of the guard which rather than use the traditional dream techniques remains solid then disappears It plays on our knowledge and our senses particularly with the scenes concerning Orton and Halliwell and the sets particularly of their flat with Halliwells murals decorating every surface are terribly well done'

In [9]:
df['review'].str.lower() #Lowercasing pandas series

,review
0,one of the other reviewers has mentioned that ...
1,a wonderful little production the filming tech...
2,i thought this was a wonderful way to spend ti...
3,basically theres a family where a little boy j...
4,petter matteis love in the time of money is a ...
...,...
1495,this movie is terrible it was so difficult to ...
1496,the only thing serious about this movie is the...
1497,2005 was one of the best year for movies we ha...
1498,according to john fords lyrically shot fiction...


lemmatization, swr removal

In [10]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

swr=set(stopwords.words('english'))
lemmatizer=WordNetLemmatizer()

In [11]:
#tokenization
df['review']=df['review'].apply(word_tokenize)

<ipython-input-11-804bac519635>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review']=df['review'].apply(word_tokenize)


In [12]:
#stopword removal
df['review']=df['review'].apply(lambda x: [word for word in x if word not in swr])
#lemmatization
df['review']=df['review'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])

<ipython-input-12-f2e03253edb4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review']=df['review'].apply(lambda x: [word for word in x if word not in swr])
<ipython-input-12-f2e03253edb4>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review']=df['review'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])


In [13]:
df['review']=df['review'].apply(lambda x: ' '.join(x))

<ipython-input-13-f1e1e2431600>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review']=df['review'].apply(lambda x: ' '.join(x))


Feature engineering

In [14]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
X=df['review']
y=df['sentiment']
y=le.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Bert Encodings

In [15]:
preprocessor=hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3')
encoder=hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4')

def get_bert_embeddings(texts):
    text_input = tf.constant(texts)  #Convert input texts to TensorFlow constants
    print(text_input)
    preprocessed_text = preprocessor(text_input)
    #print(preprocessed_text)
    outputs = encoder(preprocessed_text)
    #print(outputs)
    pooled_output = outputs['pooled_output']
    #print(pooled_output)
    return pooled_output
train_embeddings = get_bert_embeddings(X_train)
test_embeddings = get_bert_embeddings(X_test)
print(train_embeddings)

tf.Tensor(
[b'For first ever debut film riveting chilling moment In best horror film fashion pit stomach tightens every moment film The ending superb The maker Blaire Witch obviously watched film ending wasnt end beginning end A great movie piece Japans great far scare factor perfect score make think scared mind'
 b'Sixth escapade Freddy Krueger finally managed kill virtually every youth Springwood want broaden horizon SPOILER need family member order itA failure horror movie simply aint scary Works better dark macabre black comedy tell truth Freddy Krueger stripped ability chill viewer Too many wisecrack thats sure The actor arent interesting save Robert Englund always obviously slumming Yaphet Kotto simply far many visual effect The finale OK doesnt provide many spark I think one might hopeIn adding new twist familiar dream killer story provides Englund opportunity nonmakeup scene ever beforeThere cameo worth noting joint cameo thencouple Roseanne Tom Arnold devoid entertainment valu

Train Random forest classifier

In [16]:
#Train classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(train_embeddings, y_train)

#Test
y_pred = rf_classifier.predict(test_embeddings)

#Evaluate
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

In [17]:
print("Accuracy:", accuracy)
print("Confusion Matrix:\n", conf_matrix)

Accuracy: 0.67
Confusion Matrix:
 [[ 89  50]
 [ 49 112]]
